<a href="https://colab.research.google.com/github/HyeonhoonLee/NIPA2020/blob/main/tfgpt2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow

# 1. EDA

In [ ]:
import numpy as np 
import pandas as pd
import sklearn 
from sklearn import model_selection
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
os.getcwd()

In [ ]:
# DATA_IN_PATH = '/home/workspace/data/.train/.task156/data/'

In [ ]:
print("파일 크기 : ")
for file in os.listdir(DATA_IN_PATH):
    if 'tsv' in file :
        print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000, 2)) + 'MB')

In [ ]:
# randsom seed
seed=1234
np.random.seed(seed)

In [ ]:
num_classes=4

In [ ]:
#loading csv data
all_data = pd.read_csv(DATA_IN_PATH + 'train.tsv', sep = '\t', quoting = 2)
all_data.head()

In [ ]:
# all_data의 전체 길이
len(all_data)

In [ ]:
all_data['tag'].value_counts()

In [ ]:
# tag format
# 0(음악), 1(퀴즈), 2(사연), 3(수다)

In [ ]:
#stratififed split to train_data, validate_data
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

for train_idx, validate_idx in split.split(all_data, all_data["tag"]):
    train_data = all_data.loc[train_idx]
    validate_data = all_data.loc[validate_idx]

print('전체 학습데이터의 개수: {}'.format(len(train_data)))
print('전체 학습데이터의 개수: {}'.format(len(validate_data)))

In [ ]:
train_length = train_data['comment'].astype(str).apply(len)
train_length.head()

In [ ]:
# 그래프에 대한 이미지 사이즈 선언
# figsize: (가로, 세로) 형태의 튜플로 입력
plt.figure(figsize=(12, 5))
# 히스토그램 선언
# bins: 히스토그램 값들에 대한 버켓 범위
# range: x축 값의 범위
# alpha: 그래프 색상 투명도
# color: 그래프 색상
# label: 그래프에 대한 라벨
plt.hist(train_length, bins=200, alpha=0.5, color= 'r', label='word')
plt.yscale('log', nonposy='clip')
# 그래프 제목
plt.title('Log-Histogram of length of comment')
# 그래프 x 축 라벨
plt.xlabel('Length of comment')
# 그래프 y 축 라벨
plt.ylabel('Number of comment')

In [ ]:
print('comment 길이 최대 값: {}'.format(np.max(train_length)))
print('comment 길이 최소 값: {}'.format(np.min(train_length)))
print('comment 길이 평균 값: {:.2f}'.format(np.mean(train_length)))
print('comment 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('comment 길이 중간 값: {}'.format(np.median(train_length)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('comment 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('comment 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

In [ ]:
plt.figure(figsize=(12, 5))
# 박스플롯 생성
# 첫번째 파라메터: 여러 분포에 대한 데이터 리스트를 입력
# labels: 입력한 데이터에 대한 라벨
# showmeans: 평균값을 마크함

plt.boxplot(train_length,
             labels=['counts'],
             showmeans=True)

In [ ]:
# 보통은 문자열이 아닌 데이터는 삭제한다.. 숫자도 필요없을듯
train_comment = [comment for comment in train_data['comment'] if type(comment) is str]

In [ ]:
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(20, 3)
sns.countplot(train_data['tag'])

In [ ]:
for i in range(num_classes):
  print("comment 개수: {}".format(train_data['tag'].value_counts()[i]))

In [ ]:
# 각 comment 서술의 단어 수를 확인.
# 데이터를 띄어쓰기 기준으로 나눠서 그 개수를 하나의 변수로 할당한다.
train_word_counts = train_data['comment'].astype(str).apply(lambda x:len(x.split(' ')))

In [ ]:
plt.figure(figsize=(15, 10))
plt.hist(train_word_counts, bins=50, facecolor='r',label='train')
plt.title('Log-Histogram of word count in comment', fontsize=15)
plt.yscale('log', nonposy='clip')
plt.legend()
plt.xlabel('Number of comment', fontsize=15)
plt.ylabel('Number of comment', fontsize=15)

In [ ]:
print('comment 단어 개수 최대 값: {}'.format(np.max(train_word_counts)))
print('comment 단어 개수 최소 값: {}'.format(np.min(train_word_counts)))
print('comment 단어 개수 평균 값: {:.2f}'.format(np.mean(train_word_counts)))
print('comment 단어 개수 표준편차: {:.2f}'.format(np.std(train_word_counts)))
print('comment 단어 개수 중간 값: {}'.format(np.median(train_word_counts)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('comment 단어 개수 제 1 사분위: {}'.format(np.percentile(train_word_counts, 25)))
print('comment 단어 개수 제 3 사분위: {}'.format(np.percentile(train_word_counts, 75)))

In [ ]:
# 특수문자 확인하기
# 본 데이터는 크롤링 과정에서 이모티콘들이 모두 ? 로 처리되었으므로 주의
qmarks = np.mean(train_data['comment'].astype(str).apply(lambda x: '?' in x)) # 물음표가 구두점으로 쓰임
fullstop = np.mean(train_data['comment'].astype(str).apply(lambda x: '.' in x)) # 마침표
                  
print('물음표가있는 질문: {:.2f}%'.format(qmarks * 100))
print('마침표가 있는 질문: {:.2f}%'.format(fullstop * 100))

# 2. Data preprocessing

In [ ]:
# 기본 데이터
train_data.head()

## 2.1. 특수 문자 및 불용어 제거

In [ ]:
# 물음표, 마침표 등을 지움.
import string
string.punctuation

In [ ]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
train_data['comment_clean']=train_data['comment'].apply(lambda x: remove_punctuation(x))
train_data.head()

In [ ]:
#여기는 "띄워쓰기가 가능한 어절(단어)" 중에서, 분류에 영향을 미치지 않을 단어들에 대해서 작성!
stopwords = ['안녕하세요']

In [ ]:
def remove_stopwords(text):
    splited = text.split(" ")
    no_stop=[words for words in splited if words not in stopwords]
    words_wo_stop=' '.join(no_stop)
    return words_wo_stop
train_data['comment_clean']=train_data['comment_clean'].apply(lambda x: remove_stopwords(x))
train_data.head()

In [ ]:
#여기는 "음절" 중에서, 분류에 영향을 미치지 않을 단어들에 대해서 작성!
#단, '음절'이기 때문에 우리에게 필요한 어절(단어)에 들어있는 음절도 지워지게 되면 오히려 악영향을 미칠 수 있으므로, 확실하게 필요없는 것만 작성!
stopletters = ['ㅋ', 'ㅇㅈ', 'ㅎ', '...', 'ㅗ', 'ㄷ']

In [ ]:
def remove_stopletters(text):
    no_letter=[words for words in text if words not in stopletters]
    words_wo_stop=''.join(no_letter)
    return words_wo_stop
train_data['comment_clean']=train_data['comment_clean'].apply(lambda x: remove_stopletters(x))
train_data.head(10)

## 2.2. Resampling (over/under sampling)

In [ ]:
train_data.tag.value_counts()

In [ ]:
# Class count
count_class_3, count_class_1, count_class_0, count_class_2 = train_data.tag.value_counts()

# Divide by class
df_class_0 = train_data[train_data['tag'] == 0]
df_class_1 = train_data[train_data['tag'] == 1]
df_class_2 = train_data[train_data['tag'] == 2]
df_class_3 = train_data[train_data['tag'] == 3]

In [ ]:
# random oversampling (0,2 to the number of 1)
df_class_0_over = df_class_0.sample(count_class_1, replace=True)
df_class_2_over = df_class_2.sample(count_class_1, replace=True)
df_test_over = pd.concat([df_class_0_over, df_class_1, df_class_2_over, df_class_3], axis=0)

print('Random over-sampling:')
print(df_test_over.tag.value_counts())

df_test_over.tag.value_counts().plot(kind='bar', title='Count (tag)');

In [ ]:
df_test_over.head()

## 3. GPT2를 이용한 텍스트 분류

In [ ]:
# !pip install gluonnlp

In [ ]:
# # installing transforemrs
# !pip install transformers

In [ ]:
# !wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
# !unzip -o gpt_ckpt.zip

In [ ]:
# !pip install mxnet

In [ ]:
import tensorflow as tf
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
import re
from transformers import TFGPT2Model

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import tqdm

In [ ]:
# 시각화

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

In [ ]:
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token='<unused0>',
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 20
# VALID_SPLIT = 0.1
SENT_MAX_LEN = 39

DATA_IN_PATH = '/home/workspace/data/.train/.task156/data/'
DATA_OUT_PATH = '/home/workspace/user-workspace/sbs'

In [ ]:
# 텍스트 전처리
## 여기 단게에서 해도 된다!

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

In [ ]:
# train_data = train_data[:50] # for test

train_data_sents = []
train_data_labels = []

# for train_sent, train_label in df_test_over[['comment_wo_punct', 'tag']].values:
for train_sent, train_label in df_test_over[['comment_clean', 'tag']].values:
    train_tokenized_text = vocab[tokenizer(clean_text(train_sent))]

    tokens = [vocab[vocab.bos_token]]  
    tokens += pad_sequences([train_tokenized_text], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.eos_token]]

    train_data_sents.append(tokens)
    train_data_labels.append(train_label)

train_data_sents = np.array(train_data_sents, dtype=np.int64)
train_data_labels = np.array(train_data_labels, dtype=np.int64)

In [ ]:
validate_data.info()

In [ ]:
# 문자열로 변환해줌.
validate_data['comment'].astype(str)
validate_data['tag'].astype(str)

In [ ]:
# 결측치를 문자열로 채워줌.
validate_data = validate_data.fillna('결측값')

In [ ]:
validate_data.info()

In [ ]:
# validate_data

validate_data_sents = []
validate_data_labels = []

for validate_sent, validate_label in validate_data[['comment', 'tag']].values:
    validate_tokenized_text = vocab[tokenizer(validate_sent)]

    tokens = [vocab[vocab.bos_token]]  
    tokens += pad_sequences([validate_tokenized_text], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.eos_token]]

    validate_data_sents.append(tokens)
    validate_data_labels.append(validate_label)

validate_data_sents = np.array(validate_data_sents, dtype=np.int64)
validate_data_labels = np.array(validate_data_labels, dtype=np.int64)

In [ ]:
class TFGPT2Classifier(tf.keras.Model):
    def __init__(self, dir_path, num_class):
        super(TFGPT2Classifier, self).__init__()
        
        self.gpt2 = TFGPT2Model.from_pretrained(dir_path)
        self.num_class = num_class
        
        self.dropout = tf.keras.layers.Dropout(self.gpt2.config.summary_first_dropout)
        self.classifier = tf.keras.layers.Dense(self.num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=self.gpt2.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs):
        outputs = self.gpt2(inputs)
        pooled_output = outputs[0][:, -1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits

In [ ]:
BASE_MODEL_PATH = './gpt_ckpt'
cls_model = TFGPT2Classifier(dir_path=BASE_MODEL_PATH, num_class=num_classes)

In [ ]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = tf.round(tf.keras.backend.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = tf.round(tf.keras.backend.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = tf.keras.backend.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = tf.keras.backend.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + tf.keras.backend.epsilon())

    # return a single tensor value
    return recall

def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = tf.round(tf.keras.backend.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = tf.round(tf.keras.backend.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = tf.keras.backend.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = tf.keras.backend.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + tf.keras.backend.epsilon())

    # return a single tensor value
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ tf.keras.backend.epsilon())
    
    # return a single tensor value
    return _f1score

In [ ]:
# 학습 준비하기
## f1 score 계산 및 label_smoothing을 위해 one-hot label을 사용하려면 categoricalcrossentropy를 사용해야함.

optimizer = tf.keras.optimizers.Adam(learning_rate=6e-6)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.0, reduction=tf.keras.losses.Reduction.NONE)
metric = tf.keras.metrics.CategoricalAccuracy(name='accuracy')

In [ ]:
# model compile
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, f1score], experimental_run_tf_function=False)

In [ ]:
import datetime
dt = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
# Callback checkpoint dir
model_name = "tf2_gpt2" + str(dt)
checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))

In [ ]:
# overfitting을 막기 위한 earlystop 추가
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,patience=5, mode='min')
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
# class_weight = {0.0:2.0, 1.0:1.0, 2.0:3.0, 3.0:1.0}

In [ ]:
from sklearn import preprocessing

onehot_encoder = preprocessing.OneHotEncoder()

In [ ]:
reshaped = train_data_labels.reshape(-1,1)
train_data_labels_re = onehot_encoder.fit_transform(reshaped)
train_data_labels_np = train_data_labels_re.toarray()
train_data_labels_np = np.asarray(train_data_labels_np, dtype=np.int32)

v_reshaped = validate_data_labels.reshape(-1,1)
validate_data_labels_re = onehot_encoder.fit_transform(v_reshaped)
validate_data_labels_np = validate_data_labels_re.toarray()
validate_data_labels_np = np.asarray(validate_data_labels_np, dtype=np.int32)

In [ ]:
# 학습과 eval 시작
history = cls_model.fit(train_data_sents, train_data_labels_np, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                    validation_data=(validate_data_sents, validate_data_labels_np),
                    validation_steps=(len(validate_data_labels_np)/BATCH_SIZE),
                    callbacks=[cp_callback, earlystop_callback, reduce_lr])
#                     class_weight=class_weight) 
#steps_for_epoch
print(history.history)

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

In [ ]:
plot_graphs(history, 'f1score')

## 4. Prediction and submission

In [ ]:
## To load a best weights from a saved file (.h5)
cls_model.load_weights(checkpoint_path)

In [ ]:
test_data = pd.read_csv(DATA_IN_PATH + 'test.tsv', sep = '\t', quoting = 3)
test_data.head()

In [ ]:
total_number = len(test_data["comment"])
total_number

In [ ]:
# for test

test_data_sents = []

for test_sent in test_data['comment'].values:
    test_tokenized_text = vocab[tokenizer(test_sent)]

    tokens = [vocab[vocab.bos_token]]  
    tokens += pad_sequences([test_tokenized_text], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.eos_token]]

    test_data_sents.append(tokens)

test_data_sents = np.array(test_data_sents, dtype=np.int64)

In [ ]:
len(test_data_sents)

In [ ]:
from tqdm import tqdm
taglist = []
test_length = len(test_data_sents)
loop = test_length//BATCH_SIZE
for i in tqdm(range(BATCH_SIZE)):
    result1 = cls_model(test_data_sents[loop*i:(loop*(1+i))])
    prob = tf.nn.softmax(result1, axis=-1)
    y_tag = np.argmax(prob, axis=-1)
    taglist.extend(y_tag)

In [ ]:
len(taglist)

In [ ]:
test_data['tag'] = taglist
test_data.head(10)

In [ ]:
# tag format
# 0(음악), 1(퀴즈), 2(사연), 3(수다)

In [ ]:
test_data['tag'].value_counts()

In [ ]:
submission_data = test_data["tag"]
sub_df = submission_data.to_frame()
sub_df

In [ ]:
sub_df.value_counts()

In [ ]:
from nipa.taskSubmit import nipa_submit

In [ ]:
team_id = "1429"
task_no= "156"
prediction_path = '/home/workspace/user-workspace/prediction/prediction156.tsv'
save_path = os.path.join(DATA_OUT_PATH, model_name, 'prediction156.tsv')

In [ ]:
sub_df.to_csv(prediction_path, index = False, header = False)
sub_df.to_csv(save_path, index = False, header = False)

In [ ]:
print("is file: ", os.path.isfile(prediction_path))

In [ ]:
nipa_submit(team_id=team_id,
task_no=task_no,
result=prediction_path
)